<a href="https://colab.research.google.com/github/kyi93/hello-world/blob/master/Alexnet_Tomato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install q keras==2.2.0
!pip install q tensorflow==1.10.0


In [ ]:
# Lab 11 MNIST and Deep learning CNN
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import keras
import matplotlib.pyplot as plt
import os # miscellaneous operating system interfaces
import shutil # high-level file operations
from google.colab import drive #구글 드라이브에 연결
import cv2

import os
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50

In [ ]:
def recursive_find(path, path_list):    
    # 재귀적으로 함수의 절대 경로를 출력합니다.
    # path는 탐색을 시작할 폴더, path_list는 return 해줄 절대 경로의 list입니다.
    files = os.listdir(path)

    for file in files:
        full_path = os.path.join(path, file)
        recursive_find(full_path, path_list) if os.path.isdir(full_path) else path_list.append(full_path)

    return path_list


def get_data(path):
    img = []
    label = []
    for i_path in path:
        abs_path = i_path.split('/')

        try:
          temp_img = cv2.imread(i_path, cv2.IMREAD_GRAYSCALE) / 255 if color == 1 else \
              cv2.imread(i_path, cv2.IMREAD_COLOR) / 255

          # PILIMG default가 bicubic이기 때문에 bicubic으로 한겁니다.
          temp_img = cv2.resize(temp_img, (image_h, image_w), cv2.INTER_CUBIC)

          img.append(temp_img)
          label.append(abs_path[-2])

        except:
          print("error", i_path)
          continue

    img = np.asarray(img)
    img = img.reshape(img.shape[0] ,img.shape[1], img.shape[2], color)#if color
    label = [target[i] for i in label]
    label = np.asarray(label)
    label = np_utils.to_categorical(label)
    return img, label


def make_data(path):
    path_list = []
    data_path_list = recursive_find(path, path_list)
    return get_data(data_path_list)

In [ ]:
def resnet_model(n_calss):
    base_model = ResNet50(include_top = False, input_shape=(image_h, image_w, color))
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predict = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=base_model.input, output=predict)
    return model

In [ ]:
drive.mount ('/content/drive/', force_remount = True)
img_dir = '/content/drive/My Drive/img/traning_set'

Mounted at /content/drive/


In [ ]:
image_w, image_h, color = 224, 224, 3
n_class = 15
batch_size = 24
epoch = 100

training_path = r'/content/drive/My Drive/img/training_set'
test_path = r'/content/drive/My Drive/img/test_set'

target = dict([[name, _] for _, name in enumerate(os.listdir(training_path))])



In [ ]:
X, Y = make_data(training_path)
X_Train, X_Val, Y_Train, Y_Val = train_test_split(X, Y, stratify=Y, test_size=0.2)

error /content/drive/My Drive/img/training_set/Tomato_D07J/Tomato_D07_46_1979008.png
error /content/drive/My Drive/img/training_set/Tomato_D09J/Tomato_D09_46_1891876.png


In [ ]:
model = resnet_model(n_class)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=model, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# model.fit(X_Train, Y_Train, epochs=epoch, batch_size=batch_size, validation_data=(X_Test, Y_Test),
#           callbacks=[early_stopping, checkpointer, tb_hist], shuffle=True)

model.fit(X_Train, Y_Train, epochs=epoch, batch_size=batch_size, validation_data=(X_Val, Y_Val),
          callbacks=[early_stopping, checkpointer], shuffle=True)

X_Test, Y_Test = make_data(test_path)

Y_Pred = model.predict(X_Test)
Y_Test_class = np.argmax(Y_Test, axis=1)
Y_Pred_class = np.argmax(Y_Pred, axis=1)

print(classification_report(Y_Test_class, Y_Pred_class))
print(confusion_matrix(Y_Test_class, Y_Pred_class))


NameError: ignored

In [ ]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 512, 512, 3])   # img 512x512x3 (R/G/B)
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 512, 512, 1)
W1 = tf.Variable(tf.random_normal([11, 11, 3, 96], stddev=0.01)) #filter11x11x3(R,G,B)x96
#    Conv     -> (?, 512, 512, 96)
#    Pool     -> (?, 252, 252, 96)

L1 = tf.nn.conv2d(X_img, W1, strides=[1, 4, 4, 1], padding='valid')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1],
                    strides=[1, 2, 2, 1], padding='valid')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
'''
Tensor("Conv2D:0", shape=(?, 512, 512, 96), dtype=float32)
Tensor("Relu:0", shape=(?, 512, 512, 96), dtype=float32)
Tensor("MaxPool:0", shape=(?, 252, 252, 96), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 252, 252, 96), dtype=float32)
'''

# L2 ImgIn shape=(?, 252, 252, 96)
W2 = tf.Variable(tf.random_normal([5, 5, 96, 256], stddev=0.01)) # filter size 5x5 filter chaner 256
#    Conv      ->(?, 252, 252, 96)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='2')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 3, 3, 1],
                    strides=[1, 2, 2, 1], padding='valid')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)
'''

# L3 ImgIn shape=(?, 7, 7, 64)
W3 = tf.Variable(tf.random_normal([3, 3, 256, 384], stddev=0.01))
#    Conv      ->(?, 7, 7, 128)
#    Pool      ->(?, 4, 4, 128)
#    Reshape   ->(?, 4 * 4 * 128) # Flatten them for FC
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='1')
L3 = tf.nn.relu(L3)

W4 = tf.Variable(tf.random_normal([3, 3, 384, 384], stddev=0.01))
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='1')
L4 = tf.nn.relu(L4)

W5 = tf.Variable(tf.random_normal([3, 3, 384, 256], stddev=0.01))
L5 = tf.nn.conv2d(L4, W4, strides=[1, 1, 1, 1], padding='1')
L5 = tf.nn.relu(L5)
L5 = tf.nn.max_pool(L5, ksize=[1, 3, 3, 1], strides=[
                    1, 2, 2, 1], padding='valid')
L5 = tf.nn.dropout(L5, keep_prob=keep_prob)
L5_flat = tf.reshape(L5, [-1, 256 * 4 * 4])
'''
Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 2048), dtype=float32)
'''

# L6 FC 4x4x256 inputs -> 625 outputs
W6 = tf.get_variable("W6", shape=[256 * 4 * 4, 311],
                     initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([311]))
L6 = tf.nn.relu(tf.matmul(L5_flat, W6) + b6)
L6 = tf.nn.dropout(L6, keep_prob=keep_prob)
'''
Tensor("Relu_3:0", shape=(?, 625), dtype=float32)
Tensor("dropout_3/mul:0", shape=(?, 625), dtype=float32)
'''

# L7 Final FC 625 inputs -> 10 outputs
W7 = tf.get_variable("W7", shape=[311, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L6, W7) + b7
'''
Tensor("add_1:0", shape=(?, 10), dtype=float32)
'''

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy

# if you have a OOM error, please refer to lab-11-X-mnist_deep_cnn_low_memory.py

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()